# Análisis frecuencial y tiempo-frecuencia
`
Autores:
Brigitte Aguilar, Sofía Poux, Elizabeth Young
`

Modificado de *PracticalMEEG2022: MNE-python hands-on tutorial*. Por Britta Westner. 


El objetivo de este tutorial es analizar el contenido espectral de los datos. Trabajaremos en épocas.

In [1]:
# Importamos las librerías que utilizaremos a lo largo del tutorial.

%matplotlib inline
import os

import numpy as np
import matplotlib.pyplot as plt

import mne
from mne.time_frequency import tfr_morlet

Primero seteamos la dirección dónde se encuentran los datos, luego cargamos las épocas.

In [2]:
mne.set_log_level('error')

# Cambiar el data_path a la ubicación en disco donde se encuentre el archivo. Este archivo se creó en el tutorial de pre-procesamiento (1).

data_path = os.path.expanduser("D:/Usuarios/Downloads/Tutoriales_traducidos")

epochs_fname = os.path.join(data_path,
    'sub-01_ses-meg_task-facerecognition_run-01_proc-sss-epo.fif')

In [3]:
epochs = mne.read_epochs(epochs_fname, proj=True)

In [ ]:
# Visualizamos la información contenida en la estructura.

epochs.info

## Análisis frecuencial

Empezamos explorando el contenido frecuencial de las épocas que cargamos.

Primero vamos a chequear el espectro de potencia de los diferentes canales promediando a través de las épocas.

<li> El parámetro 'bandwidth' (ancho de banda) controla la resolución espectral del multitaper. Se puede incrementar la resolución eligiendo un ancho de banda más angosto a costa de un mayor tiempo de cómputo.</li>

<li> Se puede utilizar también el método 'welch', que computa el espectro de potencia sin utilizar multitapers.</li>

In [5]:
# Primero calculemos el espectro de potencia

epochs_psd = epochs.compute_psd(method='multitaper', fmin=2., fmax=40., bandwidth=2.)

# epochs_psd = epochs.compute_psd(method='welch', fmin=2., fmax=40., bandwidth=2.)

In [ ]:
# Visualizamos la información contenida en la estructura.

epochs_psd

In [ ]:
%matplotlib qt

epochs_psd.plot()

<div class="alert alert-success">
    <b>EJERCICIO</b>:
     <ul>
    <li> ¿Cuál es el nombre del canal de EEG con la mayor potencia?</li>
    <li> ¿Cómo lucen las topografías a 8-12 Hz (aproximadamente)?</li>
    </ul>
</div>

También podemos visualizar las topografías del espectro de potencia, por ejemplo, para diferentes bandas de frecuencias. En este caso, tenemos que especificar el tipo de canal que queremos visualizar (ch_type='eeg' o 'mag').

In [ ]:
%matplotlib inline
bands = {'Theta (4-8 Hz)': (4, 8), 'Alpha (8-12 Hz)': (8, 12), 
         'Beta (12-30 Hz)': (12, 30), 'Gamma (30-40 Hz)': (30, 40)}
epochs_psd.plot_topomap(ch_type='eeg', bands=bands, normalize=False, cmap='viridis');

La salida de `compute_psd()` es un objeto de tipo `Spectrum`, que se puede indexar de manera similar a `Epochs`.

A veces puede resultar interesante considerar la potencia relativa, definida como la potencia en una banda dada dividida por la potencia total. Para explorar esta opción, es necesario establecer el parámetro `normalize` a `True`.

In [ ]:
epochs_psd['face'].plot_topomap(ch_type='eeg', bands=bands, normalize=True, cmap='viridis');
epochs_psd['scrambled'].plot_topomap(ch_type='eeg', bands=bands, normalize=True, cmap='viridis');

## Análisis tiempo-frecuencia: potencia y coherencia inter-trial

Ahora calcularemos las representaciones tiempo-frecuencia (TRF, time-frequency representations) de nuestras épocas. Visualizaremos la potencia y la coherencia inter-trial (ITC). Para esto utilizaremos la función `mne.time_frequency.tfr_morlet`, pero también se puede usar `mne.time_frequency.tfr_multitaper` o `mne.time_frequency.tfr_stockwell`.

In [10]:
# Definimos las frecuencias de interés (espaciadas logarítmicamente)

freqs = np.logspace(*np.log10([5, 30]), num=20)

# Seleccionamos los canales de EEG que queremos analizar
eeg_chan=range(306,375)

n_cycles = 3.  

power, itc = tfr_morlet(epochs, freqs=freqs, n_cycles=n_cycles, use_fft=True,
                        return_itc=True, decim=3, n_jobs=1, picks=eeg_chan)


In [ ]:
# Recortamos para remover artefactos de borde

power.crop(-0.45, 1.6) 
itc.crop(-0.45, 1.6) 

### Inspeccionando la potencia

Veamos la representación tiempo-frecuencia de todos los canales. 

<div class="alert alert-info"><h4>Nota</h4><p>Las figuras generadas son interactivas. En la topografía, se puede hacer click sobre la imagen para visualizar los datos de uno de los sensores. También se puede seleccionar una porción del plano tiempo-frecuencia para obtener la topografía para esa  región determinada</p></div>




In [12]:
# Algunos ajustes de línea de base, que serán aplicados a nuestros gráficos.
baseline_mode = 'logratio' 
baseline = (-0.45, -0.1)

<div class="alert alert-info"><h4>Nota</h4><p>¡Importante! 
La línea de base con la que estamos trabajando es demasiado corta para las bajas frecuencias en nuestra descomposición:<br>
Para 5 Hz, o ventanas temporales de 3 ciclos, son 600 ms de duración. ¡Esto es más largo que que nuestro intervalo de línea de base! <br>
Además, elegimos solo 100 ms de buffer hacia 0: si centramos la ondita de 5 Hz - 3 ciclos en -100 ms, alcanzaríamos 200 ms después de 0, por lo que se estaría filtrando actividad post estímulo en nuestro baseline, lo cual no es nada bueno. <br>

Entonces, ¿qué hacemos? <br>
En la vida real, (re) cortaríamos las épocas para obtener más de 500 ms de línea de base
</p></div>

In [13]:
%matplotlib qt
power.plot_topo(baseline=baseline, mode=baseline_mode, title='Average power');

In [ ]:
%matplotlib inline
power.plot([0], baseline=baseline, mode=baseline_mode, title=power.ch_names[0]);

Nuevamente, podemos visualizar las topografías en las diferentes bandas frecuenciales

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(7, 4))
plot_freqs = [(4, 7), (8, 12), (15, 30)]
titles = ['Theta', 'Alpha', 'Beta']

for ax, freq, title in zip(axes, plot_freqs, titles):
    power.plot_topomap(ch_type='eeg', tmin=0.5, tmax=1.0, 
                   fmin=freq[0], fmax=freq[1],
                   baseline=baseline, mode=baseline_mode, 
                   axes=ax, show=False, contours=1)
    ax.set_title(title)

mne.viz.tight_layout()
plt.show()

### Gráfico conjunto

Se pueden crear también gráficos conjuntos en los cuales visualizar al mismo tiempo la TFR a lo largo de los canales y los mapas topográficos en tiempos y frecuencias específicas. De esta manera, se obtiene una descripción general de los efectos oscilatorios a través del tiempo y del espacio. 

In [ ]:
power.plot_joint(baseline=baseline, mode='mean', tmin=None, tmax=None,
                 timefreqs=[(0.15, 6.), (1., 10.)]);

### Cómputo de FWHM para nuestras onditas

Anteriormente definimos nuestras onditas en función del  _número de ciclos_. Mike X Cohen (Cohen 2019, NeuroImage (199, p. 81-86)) argumenta que es mejor definirla mediante la anchura a media altura (full-width-at-half-maximum, FWHM) de la ondita de Morlet.

Quizás recuerden que nuestra ondita de `n_cycles` es multiplicada por una Gaussiana que amortigua la ondita cerca de los bordes. Por lo tanto, FWHM es una mejor estimación del suavizado temporal que el largo total de `n_cycles`  

Veamos como hacerlo: 

In [ ]:
# Primero revisemos nuestro ciclo por frecuencia - los hemos especficado más arriba:
freqs, n_cycles  # este es el número de ciclos que pedimos por frecuencia

La fórmula para calcular la FWHM es  (Cohen 2019, eq. 4):

$ FWHM = \frac{n  \sqrt(2  \ln 2)}{\pi * f} \enspace,$

donde $n$ es el número de ciclos, $f$ es la frecuencia, y $\ln$ denota el logaritmo natural.

¡Convirtamos esta ecuación en una pequeña función!

In [18]:
def get_fwhm_morlet(n_cycles, freq):
    """Estima la FWHM de una ondita de Morlet"""

    fwhm = (n_cycles * np.sqrt(2 * np.log(2))) / (np.pi * freq)
    return fwhm

Ahora estimemos cuál es la longitud del FWHM de nuestra ondita en segundos:

In [ ]:
for freq in freqs:
    
    # estimate fwhm
    fwhm = get_fwhm_morlet(n_cycles, freq)

    # print it
    print('FWHM a %.3f Hz fue %.3f s' % (freq, fwhm))

### Inspeccionemos la coherencia inter-trial (ITC)


In [20]:
%matplotlib qt
itc.plot_topo(title='Inter-Trial coherence', vmin=0., vmax=1., cmap='Reds');

In [ ]:
%matplotlib inline
itc.plot_joint(baseline=baseline, mode='mean', tmin=None, tmax=None,
                 timefreqs=[(0.15, 6.), (1., 10.)]);